In [4]:
import pandas as pd
import numpy as np
import scipy
import scipy.sparse
import scipy.stats
import os
import scipy.io as sio
%matplotlib inline
from pylab import *

# Plotting Params:
rc('mathtext', default='regular')
fsize=14


In [5]:
f = open('snp_data/MAXENT_ALL_SD.txt')
burge_sd_scores = {}
while True:
    line =  f.readline()[:-1]
    line =  f.readline()[:-1]
    if len(line)==0:
        break
    burge_sd_scores[line.split('\t')[0]] = float(line.split('\t')[1][8:])
f.close()


In [6]:
import numpy as np
import scipy
import scipy.sparse
from pylab import *

bases = ['A','T','C','G']
dna_dict = dict(zip(list('ATCG'),range(4)))
watsoncrick = {'N':'N','.':'.','C':'G','G':'C','A':'T','T':'A','*':'*'}

def add_base(li):
		"""Used in make_mer_list to add one more base to list"""
		new_li = []
		for s in li:
			for b in bases:
				new_li.append(s+b)
		return new_li

def make_mer_list(mer_len):
	"""Makes a list of all n-mers"""
	li = bases
	for i in range(mer_len-1):
		li = add_base(li)
	return li

def reverse_complement(seq):
    outseq = ''
    for s in seq:
        outseq = watsoncrick[s] + outseq
    return outseq

def hamdist(str1, str2):
   diffs = 0
   for ch1, ch2 in zip(str1, str2):
       if ch1 != ch2:
           diffs += 1
   return diffs
        
def get_snp_pos(ref,mut):
    for p in range(len(ref)):
        if(ref[p]!=mut[p]):
            break
    return p

def make_mer_matrix_no_pos(seqs,mer_len):
    mer_dict = dict(zip(make_mer_list(mer_len),range(4**mer_len)))
    rows,cols = [],[]
    r = 0
    for i in xrange(len(seqs)):
        cur_seq = seqs[i]
        for b in range(len(cur_seq)-mer_len+1):
            rows.append(r)
            cols.append(mer_dict[cur_seq[b:b+mer_len]])
        if(r%10000)==0:
            print(r)
        r+=1
    vals = np.ones_like(cols)
    rows.append(r-1)
    cols.append(4**mer_len-1)
    vals = np.append(vals,0)
    X = scipy.sparse.csr_matrix((vals,(rows,cols)),dtype=np.float64)
    return X

def find_seq_diff_pos(wt_seq,mut_seq):
    """ Function to find the actual position of mutations based
    on the WT and MUT seq"""
    muts = np.zeros(len(wt_seq))
    for i in range(len(wt_seq)):
        muts[i] = (wt_seq[i]!=mut_seq[i])
    return find(muts)[0]

def get_snp_pos(ref,mut):
    for p in range(len(ref)):
        if(ref[p]!=mut[p]):
            break
    return p


In [7]:
# Load Splice Site Model
data = sio.loadmat('snp_data/model_full_data.mat')
sd_scores = pd.DataFrame(index=make_mer_list(6),data=data['Mer_scores'][:4**6*8].reshape(4**6,8)[:,2:6])
#exonic_mer6_scores = pd.read_pickle('../results/2015-03-11/exonic_mer6_scores.series')
#exonic_A3 = pd.read_pickle('../results/2015-03-11/exonic_acceptor_scores.series')

logit = lambda x: log(x)-log(1-x)
expit = lambda x: 1./(1.+exp(-x))

def score_seq(seq,mer_scores):
    score = 0.
    for b in range(4):
        score -= sd_scores.ix[seq[b:b+6],b]
    for b in range(0,len(seq)-5-6-3):
        score += mer_scores[seq[b:b+6]]
    # Score the SD:
    for b in range(4):
        score += sd_scores.ix[seq[len(seq)-9+b:len(seq)-9+6+b],b]
    return score

def score_maxent(seq):
    return burge_sd_scores[seq[-9:]]-burge_sd_scores[seq[:9]]

In [18]:
exonic_mer6_scores=dict()

mer6 = sio.loadmat('snp_data/mer6_series.mat')

mer_key=make_mer_list(6)
for i in range(4096):
    exonic_mer6_scores[mer_key[i]]=mer6['value'][0][i]

In [21]:
SNP_df = pd.read_pickle('snp_data/All_SNPs.df')

AttributeError: 'tuple' object has no attribute 'items'

In [20]:
SNP_df = pd.read_pickle('snp_data/All_SNPs.df')

#Get_gene_names:
header = ['chrom','splice_type','type','start','end','misc_1','strand','misc_2','ID']
A5SS = pd.read_csv('rosen/cell-2015-master/ref/hg19_v2/A5SS.hg19.gff3',sep='\t',skiprows=1,names=header)
A5SS_events = A5SS[A5SS.type=='gene']
A5SS_events['gene'] = A5SS_events.ID.apply(lambda ID:ID.split(';')[-2].split('gsymbol=')[1])
gene_names = pd.Series(dict(zip(A5SS_events.ID.apply(lambda s:s[5:].split(';')[0]),A5SS_events.gene)))
SNP_df['GENE'] = gene_names[SNP_df.EVENT].values

SNP_df = SNP_df[SNP_df.WT_SEQ.apply(len)==SNP_df.MUT_SEQ.apply(len)]
# Find the actual location of SNPs. The listed positions might be wrong:
SNP_df['SNP_POS'] = SNP_df.apply(lambda x:find_seq_diff_pos(x['WT_SEQ'],x['MUT_SEQ']),axis=1)
# Filter the SNPs
SNP_df = SNP_df[(SNP_df.SD_DIST<250) & (abs(SNP_df.WT_SEQ.apply(len)-SNP_df.MUT_SEQ.apply(len))==0) &\
                (SNP_df.SNP_POS>197) & ((SNP_df.WT_SEQ.apply(len)-SNP_df.SNP_POS)>194)]
SNP_df['WT_PSI'] = SNP_df.WT_PSI.apply(mean)
SNP_df['HETERO_PSI'] = SNP_df.HETERO_PSI.apply(mean)
SNP_df['HOMO_PSI'] = SNP_df.HOMO_PSI.apply(mean)
SNP_df['SD1_MUT'] = ((SNP_df.SNP_POS<=206) & (SNP_df.SNP_POS>197) )
SNP_df['SD2_MUT'] = (((SNP_df.WT_SEQ.apply(len)-SNP_df.SNP_POS)<=203 ) & ((SNP_df.WT_SEQ.apply(len)-SNP_df.SNP_POS)>=194) )
SNP_df['SS_MUT'] = SNP_df.SD1_MUT | SNP_df.SD2_MUT
SNP_df['ALT_EXON_MUT'] = ((SNP_df.SNP_POS>206) & ((SNP_df.WT_SEQ.apply(len)-SNP_df.SNP_POS)>203 ))

SNP_df['MUT_EFFECT_SIZE'] = SNP_df.MUT_SEQ.str.slice(198,-194).apply(lambda s:score_seq(s,exonic_mer6_scores))-\
                            SNP_df.WT_SEQ.str.slice(198,-194).apply(lambda s:score_seq(s,exonic_mer6_scores))
SNP_df['MAXENT_EFFECT_SIZE'] = SNP_df.MUT_SEQ.str.slice(198,-194).apply(lambda s:score_maxent(s))-\
                            SNP_df.WT_SEQ.str.slice(198,-194).apply(lambda s:score_maxent(s))
    
SNP_df['HOMO_PRED'] = expit(logit(SNP_df.WT_PSI)+SNP_df.MUT_EFFECT_SIZE)
SNP_df['HETERO_PRED'] = (SNP_df.WT_PSI+SNP_df.HOMO_PRED)/2.
SNP_df['HETERO_DPSI'] = SNP_df.HETERO_PSI-SNP_df.WT_PSI
SNP_df['HOMO_DPSI'] = SNP_df.HOMO_PSI-SNP_df.WT_PSI
SNP_df['HETERO_DPSI_PRED'] = SNP_df.HETERO_PRED-SNP_df.WT_PSI
SNP_df['HOMO_DPSI_PRED'] = SNP_df.HOMO_PRED-SNP_df.WT_PSI
SNP_df['LARGE_HETERO_EFFECT'] = abs(SNP_df.WT_PSI.apply(mean)-SNP_df.HETERO_PSI.apply(mean))>0.05
SNP_df['LARGE_HOMO_EFFECT'] = abs(SNP_df.WT_PSI.apply(mean)-SNP_df.HOMO_PSI.apply(mean))>0.05



AttributeError: 'tuple' object has no attribute 'items'